# Import Libraries

In [1]:
# Data
import pandas as pd
import numpy as np

# Ontology
!pip install owlready2 
from owlready2 import *
import re  # To separate words based on capital letters in onto classes & to split search queries
!pip install EMMOntoPy #Special EMMO package
# from ontopy import get_ontology


# GUI
import sys
!pip install PyQt5
import PyQt5
from functools import partial # To connect signals with methods that need to take extra arguments
    # Import 'QApplication' and all the required widgets from PyQt.QtWidgets
from PyQt5.QtWidgets import QApplication
from PyQt5.QtWidgets import QMainWindow
from PyQt5.QtWidgets import QWidget
    #layout
from PyQt5.QtCore import Qt 
from PyQt5.QtWidgets import QGridLayout # Class to create objects for arranging buttons
from PyQt5.QtWidgets import QLineEdit   # Class to create and eddit buttons and dislpay
from PyQt5.QtWidgets import QPushButton # ?
from PyQt5.QtWidgets import QVBoxLayout # Class to create layout (display, buttons)
from PyQt5.QtWidgets import QListWidget # Class to create output screen
from PyQt5.QtWidgets import QLabel      # Title
from PyQt5.QtGui import QFont           # Font
# from PyQt5.QtGui import                 #? font
from PyQt5.QtGui import QPalette
from PyQt5.QtGui import QColor


#import dash #(version .. ) pip install dash
#import dash_core_components as dcc
#import dash_html_components as html

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


TO DO: 
    1) Make all steps a function in a class of method functions
    2) Adjust the script and call all the functions and classes 

## Ontology 

In [2]:
#battinfo = get_ontology('https://github.com/BIG-MAP/BattINFO/blob/master/battinfo.ttl').load()

# write the location to the ontology to the onto_path list
onto_path = ["file:///home/hendrik/Deliverables_FAIRBatt/GUI_VersionControl/V0.2/20221025/Ontology/test.owl"]
    # onto_path[1] = <fill in path > 
    
# Import ontology from local
try:
    onto = get_ontology(onto_path[0]).load()
except:
    pass 

onto = get_ontology(onto_path[0]).load()
# Note that all the documents has to be owl files.

# Collecting classes from ontology in a list
class_list_raw = list(onto.classes())
#print("Ontology Imported")
#print(class_list_raw)
#len(class_list_raw)
#print(class_list_raw)

### Select Specific Ontology Classes 

In [3]:
# Select classes of the imported ontology 
    ## To do: Turn this box into a function
        # def select_classes( tag, class_list_raw, prefix, onto, ..ALSO package needed?..)
        # Output1 = class_list_select --> List of selected classes
        # Output2 = printed message with info on result
    
    
# Storage to store classes:
class_list_select = []

# Name tag
tag = 'ElectrochemicalFlowCell'    #Name tage
prefix = 'flowbattery.'    #prefix

# Select classes with the name tag
for j in class_list_raw:
    string = str(j).removeprefix('electrochemistry.') #remove title electrochemistry
    if string[-len(tag):] == tag: #search for the tagged classes based on length of the name tag
        if len(string) > len(tag):
            string = string.removesuffix(tag) #removes tag when it has a suffix, if it's a class no removal
            string = re.sub(r"(?<=\w)([A-Z])", r" \1", string) # Separate by capital letters
            string = re.sub('_',  ' ', string)
        class_list_select.append(string)

# print result        
print("Number of classes with assigned name tag -" 
      + tag
      + "- and prefix -"
      + prefix
      + "- is "
      + str(len(class_list_select))
      + ": \n    !Caution: tag can be an added as suffix, prefix or another part of a class name")

print("Type 'class_list_select to see the selected classes'")

Number of classes with assigned name tag -ElectrochemicalFlowCell- and prefix -flowbattery.- is 33: 
    !Caution: tag can be an added as suffix, prefix or another part of a class name
Type 'class_list_select to see the selected classes'


In [4]:
class_list_select

['Annode',
 'Anolyte Tank',
 'Aqueous Metal',
 'Aqueous Metalic Electrolyte',
 'Aqueous Organic',
 'Aqueous Organic Electrolyte',
 'C U Plate',
 'Cathode',
 'Catholyte Tank',
 'Container',
 'Current Collector',
 'Daniel Cell',
 'ElectrochemicalFlowCell',
 'Electrolyte',
 'End plate',
 'Flow Field',
 'Gasket',
 'Intermediate Plate',
 'Long List Redox Flow Batt',
 'Membrane',
 'Membraneless',
 'Non Aqueous Organic',
 'Non Aqueous Organic Electrolyte',
 'Non Aqueous Organometallic',
 'Non Aqueous Organometallic Electrolyte',
 'Power density',
 'Pump',
 'Pumpless',
 'Redox Potential',
 'Stack',
 'Subcomponent',
 'leakages',
 'test.Spacer (mesh)']

#### TO DO: Must this go in the GUI script?

In [5]:
"""Create dictionaries of buttons for later use in the interface"""

# TO DO: Make this a function

# Buttons for ontology classes
classbuttons = {} # create empty dict
keys = class_list_select # Ontology classes become keys
root = np.ceil(np.sqrt(len(class_list_select))) # number of rows and columns of the bottons
# Fill dictionary
for classnumber in range(len(keys)):
    classbuttons[keys[classnumber]] = np.divmod(classnumber, root)
    
print("Buttons with names of the ontology classes are prepared now")    

# Buttons for logic operations
logicbuttons = {} # create empty dict
logicbuttons["and"] = (0 , 0)
logicbuttons["or"] = (0 , 1 )
logicbuttons["-Clear Screen-"] = (0 , 2)
logicbuttons["Search"] = (0, 3)

print("Buttons with logic operations are prepared now")
print(logicbuttons)

Buttons with names of the ontology classes are prepared now
Buttons with logic operations are prepared now
{'and': (0, 0), 'or': (0, 1), '-Clear Screen-': (0, 2), 'Search': (0, 3)}


## Prepare Data Set

In this part the abstracts of articles in the data base are scanned for keywords. Here the ontology classes are used as the keywords.

TO DO:  
- plement pygetpapers, to automatically download the text if it is open source/public available. This text can be scanned on keywords. See: https://joss.theoj.org/papers/10.21105/joss.04451 
- Implement more advanced scanning, e.g., extract more detailed information on the keywords(ontology classes) such as the individual instances of the class mentioned in the article!

In [6]:
"""Import Zotero Data Base from Github"""
# To Do: make this a function

# Import raw data from github repository
url_flowbatt_raw = 'https://raw.githubusercontent.com/hendelhendel/FlowBattDATA/main/data/FlowBattKnowledgebase_1.csv'
df_flowbatt_raw = pd.read_csv(url_flowbatt_raw)

# select a part of the data for testing
df_flowbatt_select1 = df_flowbatt_raw[["Title","DOI", "Abstract Note"]][:]
# df_flowbatt_select1.style # show the data frame
# df_flowbatt_select1.info()  # Show info


# Add ontology classes as new columns to the data frame
for j in class_list_select:
    col = str(j)
    vec = [None] * len(df_flowbatt_select1)
    df_flowbatt_select1[col] = vec
    
#df_flowbatt_select1
#df_flowbatt_select1[["Abstract Note"]][:]
#df_flowbatt_raw

In [7]:
""" Search Abstracts data """
# Has to be a appart file and function
# TO DO: Read out whole files?
# To DO: Whole files need to be in the csv files as tekst? 

print("!! Note that the resulting data frame is NOT correct since it is a dummy set")


# Read out Abstracts
for row in range(0, len(df_flowbatt_select1)): # starts at row = 0 ends at row = leng
    
    # Select Abstract text
    abstr_test = df_flowbatt_select1['Abstract Note'].iloc[row]
    
    # Search Abstract if esixt
    if type(abstr_test) == str:
        abstr_test = abstr_test.lower() # lower all capital letters
        abstr_test_sentence = re.split("\. |\! |\? ",abstr_test) # Split abstract per sentences in a list
        #Scan text for ontology terms
        for j in class_list_select:
            
#            df_flowbatt_select1[j].iloc[row] = [] # list to store search results [yes/no , sentence/None]
            string = str(j).lower()  # write class as a string withoud capital letters to search text
            if abstr_test.find(string) != -1:
                temp_list = []
                temp_list.append('yes')
#                df_flowbatt_select1[j].iloc[row][0] = 'yes' # if ontoclass is found in abstract store yes
                for sentence in abstr_test_sentence:
                    if sentence.find(string) != -1:
                        temp_list.append(sentence)
                df_flowbatt_select1[j].iloc[row] = temp_list
#                        df_flowbatt_select1[j].iloc[row][1] = sentence # * len(df_flowbatt_select1) # write answer to the right location in the dataframe
                
            else:
                df_flowbatt_select1[j].iloc[row] = ['no', 'no']
                #print(string + ": " + str(abstr_test.find(string)) + " found in row " + str(row))
                
   # else:
   #     print("Abstract is not available for " + str(df_flowbatt_select1["Title"].iloc[row]) + " with row number " + str(row))

    
#df_flowbatt_select1
        

!! Note that the resulting data frame is NOT correct since it is a dummy set


In [8]:
df_flowbatt_raw[["Title","DOI", "Abstract Note"]][69:74]

,Title,DOI,Abstract Note
69,Redox flow cells for energy conversion,10.1016/j.jpowsour.2006.02.095,Energy storage technologies provide an alterna...
70,Evaluation of N-ethyl-N-methyl-morpholinium br...,10.1016/j.electacta.2013.06.084,Evolution of bromine gas in the positive half ...
71,"1,2-Dimethylimidazole based bromine complexing...",10.1016/j.ijhydene.2019.03.050,To stabilize bromine produced during a vanadiu...
72,Impact of membrane characteristics on the perf...,10.1016/j.jpowsour.2015.03.010,The Br2/H2 redox flow cell shows promise as a ...
73,Core-shell rhodium sulfide catalyst for hydrog...,10.1016/j.jpowsour.2018.02.005,Synthesis and characterization of high electro...


In [9]:
df_flowbatt_select1[["Membrane"]].iloc[72][0]

['yes',
 'in this paper, the effect of various aspects of material selection and processing of proton exchange membranes on the operation of the br2/h2 redox flow cell is determined',
 'membrane properties have a significant impact on the performance and efficiency of the system',
 'the impact of thickness, pretreatment procedure, swelling state during cell assembly, equivalent weight, membrane reinforcement, and addition of a microporous separator layer on this tradeoff is assessed',
 'a cell with this membrane was cycled continuously for 3164\xa0h',
 'membrane transport properties, including conductivity and bromine and water crossover, were found to decrease moderately upon cycling but remained higher than those for the as-received membrane.']

In [10]:
"""Check the number of times a specific ontological term is found in a abstract"""
# TO DO: store the number of appearence per term in data frame

#df_flowbatt_select1["Membrane"].iloc[:]
for row in range(0, len(df_flowbatt_select1)):
    if type(df_flowbatt_select1['Membrane'].iloc[row]) == list:
        vectop = [row,len(df_flowbatt_select1['Membrane'].iloc[row])]
        print(vectop)

[0, 2]
[2, 2]
[7, 2]
[8, 2]
[9, 2]
[10, 2]
[11, 2]
[13, 2]
[14, 3]
[15, 2]
[16, 2]
[17, 2]
[18, 2]
[19, 2]
[21, 2]
[22, 2]
[23, 2]
[24, 3]
[25, 2]
[26, 2]
[27, 2]
[28, 2]
[29, 2]
[30, 2]
[31, 2]
[32, 2]
[33, 2]
[34, 2]
[35, 3]
[36, 2]
[37, 2]
[38, 2]
[39, 2]
[40, 2]
[41, 2]
[42, 2]
[44, 2]
[47, 2]
[48, 2]
[49, 2]
[53, 2]
[55, 2]
[56, 2]
[57, 2]
[68, 3]
[69, 2]
[70, 2]
[71, 2]
[72, 6]
[73, 2]
[75, 2]
[76, 2]
[77, 2]
[79, 4]
[84, 2]
[85, 2]
[86, 2]
[87, 2]
[88, 2]
[90, 2]
[91, 2]
[92, 2]
[93, 2]
[94, 2]
[95, 2]
[96, 2]
[97, 2]
[101, 2]
[102, 2]
[103, 2]
[105, 2]
[106, 2]
[107, 2]
[109, 2]
[110, 2]
[111, 2]
[112, 2]
[113, 2]
[114, 2]
[115, 2]
[116, 2]
[117, 2]
[118, 2]
[119, 2]
[120, 2]
[121, 2]
[122, 2]
[123, 4]
[124, 2]
[125, 2]
[126, 2]
[127, 2]
[128, 2]
[129, 7]
[130, 2]
[131, 2]
[132, 2]
[133, 2]
[134, 2]
[135, 3]
[136, 2]
[137, 2]
[138, 2]
[140, 2]
[141, 2]
[142, 2]
[143, 4]
[144, 2]
[145, 2]
[146, 2]
[147, 2]
[148, 3]
[149, 2]
[150, 3]
[151, 2]
[152, 3]
[153, 4]
[154, 2]
[155, 2]
[

## GUI

TO DO:
- Make all buttons operatable.
- Reduce number of hard coding parts.
- Make DOI clicable, copy-able and export to CSV.
- Implement logic oporator "and" also in the output.
- Implement logic operator "or".
- Implement logic operator "=". This helps with more advanced keyword searches. 
    Example:  "Membrane = some instance" | AND| "Electrolyte = some instance" etc. 
- Implement systematic review to sort the articles on relevance
- what to do with synonyms etc per ontoclass?
-        TO DO: MAKE ALL THE SENTENCES VISABLE, TEST IT WITH ROW MEMBRANE ROW 72
         Add the text: "Membrane found in the sentence...""
         Add "ONTO class FOUNd X times in database" 
         Add "class1 AND clss 2 found C times in data base" etc.

In [ ]:

#!/usr/bin/env python3

# Filename: pycalc.py

"""Rekenmachine is a simple calculator built using Python and PyQt5."""

ERROR_MSG = 'ERROR' # Handles the errors of a calcultation by the calculator


__version__ = '0.1'
__autor__ = 'Hendrik Snijder'


# Create sub window to edit the main window
class EditWindow(QWidget):
    """ This window can be used to store the edit buttons of the main window. Such as the ontology buttons
    This Window is a Q Widget. If it has no parent it wil appear as a free-floeating window."""

    # Properties
    def __init__(self):
        super().__init__()
        layout = QVBoxLayout()
        self.label = QLabel("Edit Window")
        layout.addWidget(self.label)
        self.setLayout(layout)
    

#  VIEW: Create a subclass of QMainWindow to setup Main window of GUI
class PyCalcUi(QMainWindow):
    """VIEW of the rekenmachine(GUI). (public interface)"""
    
    # Properties
    def __init__(self):
        """View initializer."""
        super().__init__()
        # step3. Set some main window's properties
        self.setWindowTitle('Test de GUI')  #set the window title
        self.setFixedSize(1111, 731)  #set the size of the window
        
        # Set the central widget and the general layout (!keep same order!)
        self.generalLayout = QVBoxLayout() # USed to create display at top and buttons in a grid at bottom.
        self._centralWidget = QWidget(self) # Create a Qwidgit (central widget) = parent for rest of GUI
        self.setCentralWidget(self._centralWidget) 
        self._centralWidget.setLayout(self.generalLayout)
        # Create dislpayname
        #self.display.nameLabel.setText('Name:')
        
        # Create the display and the buttons
        self._createDisplay() #creates display at the top of the window(method defined below)
        self._createClassButtons() # Creates class buttons in a grid at the bottom (method defined below) hence e.g. self.buttons EXISTS!
        self._createLogicButtons() # Creates logical operator buttons
        self._createOutputScreen() # Creates outputscreen
      #  self._createNames()
        self._createResultButtons()#
        self._createWindowButtons() # Creates window buttons
        
#TO create separate window for the ontology buttons do the following: 
# Check https://www.pythonguis.com/tutorials/creating-multiple-windows/
#        self._createOpenEditwindow() # Create button to open edit win
        
        
        # Create Edit window
#        self.editwindow = EditWindow()
#        editwindowBTN = QPushButton("Open Edit Window")
#        editwindowBTN.clicked.connect(self.openEditWindow)
#        self.editwindow.addWidget(editwindowBTN)
        
    def openEditWindow(self, checked):
        W = EditWindow()
        W.show

 
    # Methods: Create View Elements
    def _createNames(self):
        """Create names"""
        self.nameLabel = QLabel(self)
        self.nameLabel.setText('Query')
        self.nameLabel.setFont(QFont("Query", 17, QFont.Bold))
     #   self.nameLabel.move(20, 29)
    
    def _createDisplay(self): 
        """Create the display."""
        # Create the display widget
        self.display = QLineEdit() # Create an object in class QLineEdit
        # Set some display's properties
        self.display.setFixedHeight(100) #fix display pixels
        self.display.setAlignment(Qt.AlignRight) #D isplay shows text as left alined
        self.display.setReadOnly(False) # Read-only to avoid direct editing in the display
        self.display.setFont(QFont("Query", 13, QFont.Bold))
        self.setDisplayText('Enter Query...')
  
        #Add the display to the general layout
        self.generalLayout.addWidget(self.display)
        
    def _createClassButtons(self):
        """Create Buttons"""
        self.buttons = {} #create empty dictionary to hold calculator buttons
        buttonsLayout = QGridLayout() # Create a temporary dictionary to store labels and positions (an object in QGridLayout)
        # Button text | position on the QGridLayout
        buttons = classbuttons
                  
        # Create the buttons and add them to the grid layout in a for loop
        for btnText, pos in buttons.items():  # items is a function for dictionary (e.g. buttons)
            self.buttons[btnText] = QPushButton(btnText)
            self.buttons[btnText].setFixedSize(173,45)  #fixed size of button
            buttonsLayout.addWidget(self.buttons[btnText],pos[0], pos[1])
        # Add buttonsLayout to the general layout
        self.generalLayout.addLayout(buttonsLayout)

    def _createLogicButtons(self):
        """Create Buttons"""
        self.logicbuts = {} #create empty dictionary to hold calculator buttons
        logicbutsLayout = QGridLayout() # Create a temporary dictionary to store labels and positions (an object in QGridLayout)
        # Button text | position on the QGridLayout
        logicbuts = logicbuttons
                  
        # Create the buttons and add them to the grid layout in a for loop
        for Text, pos in logicbuts.items():  # items is a function for dictionary (e.g. buttons)
            self.logicbuts[Text] = QPushButton(Text)
            self.logicbuts[Text].setFixedSize(100,45)  #fixed size of button
            logicbutsLayout.addWidget(self.logicbuts[Text],pos[0], pos[1])
        # Add buttonsLayout to the general layout
        self.generalLayout.addLayout(logicbutsLayout)       
        
    def _createOutputScreen(self):
        """Create Output list"""
        # Create outputscreen widget
        self.outputscreen = QListWidget() # Create an object in de class QlistWidget
        # Set some output screen properties
        
        # Add outputscreen to the general layout
        self.generalLayout.addWidget(self.outputscreen)
        
        # Extra buttons in the OutputScreen
        #  gotobtn = QPushButton("Go To",self)
        #gotobtn.clicked.connect(self.o)
    
    def _createResultButtons(self):
        """Create Go To Button"""
        self.resultbuttons = {} # Create empty dictionary to hold result buttons
        resultbuttonsLayout = QGridLayout() # Create a temporary dictonary to store labels and positions 
        # result buttons test| position
        resultbuttons = {"Go To": (0,0),
                        "Copy": (0,1),
                         "Export to CSV": (0,2)
                        }
        for text, pos in resultbuttons.items():
            self.resultbuttons[text] = QPushButton(text)
            self.resultbuttons[text].setFixedSize
            resultbuttonsLayout.addWidget(self.resultbuttons[text],pos[0], pos[1])
        # Add resultbuttonsLayout to general layout
        self.generalLayout.addLayout(resultbuttonsLayout)
    
    def _createWindowButtons(self):
        """Creat window buttons"""
        self.editwindowBTN = {} # Create empty dictionary
        editwindowBTNLayout = QGridLayout()
        editwindowBTN = {"Push for window 1": (0,0),
                        "Push for window 2": (0,1),
                         "Push for window 3": (0,2)
                        }
        for text, pos in editwindowBTN.items():
            self.editwindowBTN[text] = QPushButton(text)
            self.editwindowBTN[text].setFixedSize
            editwindowBTNLayout.addWidget(self.editwindowBTN[text],pos[0], pos[1])
        # Add buttons to layout   
        self.generalLayout.addLayout(editwindowBTNLayout)
        # Connect button to a specific action
        self.editwindowBTN['Push for window 1'].clicked.connect(self.openEditWindow)
            
              
#       self.editwindowBTN = QPushButton("Push for Window")
#       self.button.clicked.connect(self.)
#       self.generalLayout.addLayout(self.editwindowBTN) 
    
    
    # Methods: update Display View
    def setDisplayText(self, text):
        """Set display's text"""
#        text = text.replace('Enter Query...', '')
        self.display.setText(text) # Set and update the display's 'text'
        self.display.setFocus() # Set the cursor's focus on the display
        
    def displayText(self):
        """Get display's text"""
        return self.display.text() # Return display's current text (used as the math expr. to be evaluated.)
    
    def clearDisplay(self):
        """Clear the display"""
        self.setDisplayText('Enter Query...') # Set input screen back to 'Enter Query...' screen again keep it () to fully ')

         
    # Methods: Update outputscreen
    def setOutputScreenText(self, text): 
        """Set text outputscreen"""
        self.outputscreen.addItems(text)

    def clearOutputScreen(self):
        """Clear the outputscreen"""
        self.outputscreen.clear() #clears outputscreen 
        
        
# CONTROLLER: Create a Controller class to connect the GUI and the model
    # Controller class needs to perform ~ 3 tasks:
        # 1. acces the GUI public interface
        # 2. Handle the creation of math expressions
        # 3. Connect butten 'clicked' signals with the appropriate slots

class PyCalcCtrl:
    """PyCalc Controller class."""
    
    # Properties
    def __init__(self, model, view):
        """Controller initializer."""
        self._evaluate = model
        # Give an object of the vieuw PyCalcUi. 
            # Note: in main() view is defines as an object of PyCalcUi, hence all methods and properties of it applies!
            # Gives Full acces to the public interface
        self._view = view # imports view as a property
        # Connect signals and slots
        self._connectSignals()  # method defined below

        
    # Methods
    def _calculateResult(self):
        """Evaluate expressions."""
        # Evaluate the expression entered in the display by the user ('=' is not displayed!)
        que = self._view.displayText()  # Input que
        parts = que.split('| AND |')  # Split que in ontoclasses
        df = df_flowbatt_select1 # Data frame to search
        answer = 'Search Results: '  # Create string to store result
#        count = [None] * len(parts) # storage to count number of times an onto class is found
        for part in parts: # if part is not a class in the onto, an error arise!
#           storage = {}  --> Here we can store the result per part, so that we can add the logical opporators to the answers
#            storage.append()
            for row in range(0,len(df)):  # Select rows to enter values of onto class
                try: # try if search term is in database
                    if type(df[part].iloc[row]) == list: # Check if Onto class has a value (i.e. if it is not None)
                        if type(df["DOI"].iloc[row]) == str and df[part].iloc[row][0] == 'yes':
    #                        count = count + 1
                            answer = answer + "\n   " + part + " found in the abstract of DOI:" + df["DOI"].iloc[row] + "  in the sentence(s)"
                            for sentence in range(1,len(df[part].iloc[row])):
                                answer = answer + "\n           " + df[part].iloc[row][sentence] # store DOI of article found
                except KeyError: # if search term is not in database no search can be done
                    answer = part + ' Not found in data base'

    
        ###COUNtCOUNTCOUNTCOUNTCOUNT!!!
        # Add "ONTO class FOUNd X times in database" 
        # Add "class1 AND clss 2 found C times in data base" etc.
                    
        # Show anwser 
        result = answer
        # self._view.setDisplayText(result) # disabled to see input query on display
        self._view.setOutputScreenText(result.split("\n"))
    
    def _buildExpression(self, sub_exp):
        """Build expression.""" 
        # Clear display when error occurs
        if self._view.displayText() == ERROR_MSG:
            self._view.clearDisplay()
            
        # To handle the creation of expressions
        if sub_exp == 'and': 
            expression = self._view.displayText() + '| AND |'
            print('and')
        elif sub_exp == 'or':
            expression = self._view.displayText() + '| OR |'
        else: 
            expression = self._view.displayText() + sub_exp # expression to be evaluated + newest input
            expression = expression.replace('Enter Query...', '') # remove prefix Enter Query... after pressing a button
        self._view.setDisplayText(expression) # Uses function defined above, to display user input

    def _connectSignals(self):
        """Connect signals and slots."""
        # Connect the printable buttons with _buildExpression()
        for btnText, btn in self._view.buttons.items():  # for (a, b) in - buttons-list of (key, value) 
            if btnText not in {'-Clear Screen-','Search'}:
                btn.clicked.connect(partial(self._buildExpression, btnText))
        
        # Logic button AND 
    # self._view.logicbuts["and"].clicked.connect(partial(self._buidExpression, 'and'))
        
        # Logic buttons
        for logicbtnText, logicbtn in self._view.logicbuts.items():
            if logicbtnText not in {'-Clear Screen-','Search'}:
                logicbtn.clicked.connect(partial(self._buildExpression, logicbtnText))
        
        # connect 'search' button to calculate. check: (?) clicked.connect() is method of objects in PyCalcUi (?)
        self._view.logicbuts['Search'].clicked.connect(self._calculateResult) 
        # connect calculated result to display. returnPressed.connect() is method of objects in class QLineEdit
        self._view.display.returnPressed.connect(self._calculateResult) 
        # connect 'C' button to clear display
        self._view.logicbuts['-Clear Screen-'].clicked.connect(self._view.clearDisplay) 
        self._view.logicbuts['-Clear Screen-'].clicked.connect(self._view.clearOutputScreen)
        
        # connect Go To button
        
# MODEL: Create a function for the model to handle the calculator's operation
def evaluateExpression(expression):
    """Evaluate an expression."""
    try:
        # Use eval() to evaluate a string as an expression 
        result = str(eval(expression, {}, {})) # !IMPORTANT: only use eval() on trusted input due to security issues!!!
    except Exception: 
        result = ERROR_MSG
        
    return result
        

# Client code: Gives the actually routine!
def main(): # Defines the calculator's mainQ function (= best practise)
    """Main function."""
    # step 2. Create an instance of QApplication
    pycalc = QApplication(sys.argv) #creates object pycalc
    # Force the style to be the same on all OSs:
    pycalc.setStyle("Fusion")
    
    # Now use a palette to switch to dark colors:
    palette = QPalette()
    palette.setColor(QPalette.Window, QColor(53, 53, 53))
    palette.setColor(QPalette.WindowText, Qt.white)
    palette.setColor(QPalette.Base, QColor(53, 53, 53))
    palette.setColor(QPalette.AlternateBase, QColor(53, 53, 53))
    palette.setColor(QPalette.ToolTipBase, Qt.black)
    palette.setColor(QPalette.ToolTipText, Qt.white)
    palette.setColor(QPalette.Text, Qt.white)
    palette.setColor(QPalette.Button, QColor(53, 53, 53))
    palette.setColor(QPalette.ButtonText, Qt.white)
    palette.setColor(QPalette.BrightText, Qt.red)
    palette.setColor(QPalette.Link, QColor(42, 130, 218))
    palette.setColor(QPalette.Highlight, QColor(42, 130, 218))
    palette.setColor(QPalette.HighlightedText, Qt.black)
    pycalc.setPalette(palette)
    
    #step 4. Show the calculator's GUI
    view = PyCalcUi() # vieuw is object of class PyCalcUi
    view.show()  #shows the GUI
    # Create instances of the model and the controller
    model = evaluateExpression
    PyCalcCtrl(model=model, view=view)  # view is now also object of class PyCalcCtrl!
    #step5. Execute the calculator's main loop
    sys.exit(pycalc.exec_())  #Wruns application's event loop

if __name__ == '__main__': #This code will make sure that python execute the function main first. (See below)
    main()

#In this code, there is a function called main() that prints the phrase Hello World! when the Python interpreter executes it. 
#There is also a conditional (or if) statement that checks the value of __name__ and compares it to the string "__main__". 
#When the if statement evaluates to True, the Python interpreter executes main(). 
#You can read more about conditional statements in Conditional Statements in Python.

/tmp/ipykernel_2405709/2691637952.py:106: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  buttonsLayout.addWidget(self.buttons[btnText],pos[0], pos[1])


In [ ]:
testentestetestetest = {'7': (0, 0),
                   '8': (0, 1),
                   '9': (0, 2),
                   '/': (0, 3),
                   'C': (0, 4),
                   'Hendrik': (0,5),
                   '4': (1, 0),
                   '5': (1, 1),
                   '6': (1, 2),
                   '*': (1, 3),
                   '(': (1, 4),
                   'is': (1,5),
                   '1': (2, 0),
                   '2': (2, 1),
                   '3': (2, 2),
                   '-': (2, 3),
                   ')': (2, 4),
                   'de': (2,5),
                   '0': (3, 0),
                   '00': (3, 1),
                   '.': (3, 2),
                   '+': (3, 3),
                   '=': (3, 4),
                   'BESTE': (3,5),
                   
                  }

In [ ]:
check =  {"Go To": (0,0),
                        "Copy": (0,1)}
for a, b in check.items():
   # print[a]
    print(b[1])
check.items()

#########################################################3
#DOI link kan je openen door al url te nemen:
    #https://doi.org/ vul hier DOI in. 

In [ ]:
testlijst = []
testlijst.append("1")
testlijst.append("hond")

In [ ]:
testlijst

In [ ]:
string = "Mebrane| AND |Electrolyte| OR |Voltage| AND |Separator| OR |Firkandel| AND |Frietjes| AND |Mayonaise| OR | Appelsap"
#tring = re.sub(r"(?<=\w)([A-Z])", r" \1", string) # Separate by capital letters
string = string.split('| OR |')
print(string)
for j in range(len(string)):
    string[j]  = string[j].split('| AND |')
    print(string)


In [ ]:
testtext = 5
print(testtext)

In [ ]:
testtext = 'Enter Query...Membrane| AND |Electrolyte'
print(testtext)
testtext = testtext.replace('Enter Query...', '')
print(testtext)
testtext = testtext.replace('Enter Query...', '')
print(testtext)